In [1]:
import sys 
sys.path.append('../..')

In [2]:
import zarr 

In [3]:
from main_utils import LLM_EmbedData

In [64]:
data = LLM_EmbedData(
    data_source_yaml_path='../../datasets/product_title_embedding/wish-mturk-test.yaml', 
    model_name='sentence-transformers/sentence-t5-large'
)

In [65]:
data.hparams.raw_cache_dir_folder

'/data/junwang/.cache/general/15ad488cde7aadf086c3366823defd0d'

In [66]:
!ls /data/junwang/.cache/general/b9229326675dc973600739490633877a

test.json.gz


In [67]:
import pandas as pd
import os

In [68]:
df = pd.read_json(
    os.path.join(data.hparams.raw_cache_dir_folder, 'test.json.gz'), 
    lines=True)

In [69]:
df

,text_input,text_output
0,Embed product: Fantasy Master - 681 - sv√§rd,Embed taxonomy: education & office supplies > ...
1,Embed product: Silent Red Thing Silent Hill Py...,Embed taxonomy: home & garden > home textile >...
2,Embed product: Handmade Leather Goods Making B...,Embed taxonomy: education & office supplies > ...
3,Embed product: New Upgrade 4/2/1 Large Solid C...,Embed taxonomy: home & garden > home textile >...
4,Embed product: 20pcs/set DIY Full Cover False ...,Embed taxonomy: beauty & health > nails art & ...
...,...,...
6396,Embed product: Kuromi Strawberry Picking Straw...,Embed taxonomy: men's clothing > tops & tees >...
6397,Embed product: Women's European and American F...,Embed taxonomy: luggage & bags > women's bags ...
6398,Embed product: Grumpy Old Pontiac Trans AM Own...,Embed taxonomy: home & garden > home decor > p...
6399,"Embed product: USA SELLER FAST SHIPPING 9.5"" T...","Embed taxonomy: home & garden > kitchen,dining..."


In [70]:
df_infer = pd.read_json('../../models/product_title_embedding/version_0/epoch=4-step=370604--wish-mturk-test.json', lines=True)

In [71]:
df_infer

,embedding,batch_idx,dataloader_idx,batch_indices
0,"[1.105878710746765, 1.5727436542510982, -0.699...",0,0,0
1,"[-0.10123966634273501, 1.385772824287414, -0.5...",0,0,1
2,"[0.614453554153442, 0.45487579703330905, 1.220...",0,0,2
3,"[0.322158545255661, -0.45887371897697404, -0.6...",0,0,3
4,"[0.596618175506591, -0.717659294605255, -1.495...",0,0,4
...,...,...,...,...
6396,"[0.155938535928726, 1.799080371856689, 0.54478...",399,0,6396
6397,"[0.7631920576095581, 0.37477967143058705, -0.7...",399,0,6397
6398,"[0.172190308570861, -0.008893728256225001, -0....",399,0,6398
6399,"[-0.7516044974327081, 0.649876952171325, 0.006...",399,0,6399


In [72]:
z = zarr.open('../../models/product_title_embedding/version_0/epoch=4-step=370604--wish-mturk-test.zarr', 'r')

In [73]:
list(z.array_keys())

['dl-idx_b-idx_s-idx_pred_rank-0']

In [74]:
import numpy as np

In [75]:
z_arr = np.concatenate([z[i][:] for i in list(z.array_keys())])

In [76]:
z_arr.shape

(6401, 1027)

In [77]:
s_idx = z_arr[:, 2].astype(int)
embs = z_arr[:, 3:]

In [78]:
assert len(s_idx) == len(set(s_idx))

In [79]:
embs = embs[np.argsort(s_idx)]

In [80]:
embs

array([[ 1.10587871,  1.57274365, -0.699646  , ..., -0.53345925,
         0.87789041,  0.46132916],
       [-0.10123967,  1.38577282, -0.54628152, ...,  0.41911453,
         1.451998  , -0.42471376],
       [ 0.61445355,  0.4548758 ,  1.22053003, ...,  0.83871603,
         0.29160708,  0.79073805],
       ...,
       [ 0.17219031, -0.00889373, -0.83663177, ...,  0.60282713,
        -0.52359337, -0.53481078],
       [-0.7516045 ,  0.64987695,  0.00639064, ...,  1.45288086,
        -0.12274542, -1.62278986],
       [ 1.74298954,  0.550556  , -1.12343776, ..., -1.09264231,
        -0.44712359, -0.19076888]])

In [81]:
df_infer[df_infer.batch_indices == 0]

,embedding,batch_idx,dataloader_idx,batch_indices
0,"[1.105878710746765, 1.5727436542510982, -0.699...",0,0,0


In [83]:
import umap

In [84]:
mapper = umap.UMAP().fit(embs)

In [85]:
import umap.plot

In [86]:
umap.plot.output_notebook()

Loading BokehJS ...

In [92]:
hover_data = pd.DataFrame({'index': np.arange(len(df)),
                           'label': df['text_output'],
                           'text': df['text_input']
                           })

In [93]:
fig = umap.plot.interactive(
    mapper, 
    labels=df['text_output'].apply(lambda x: x.split(': ')[1].split(' > ')[0]),
    point_size=5,
    hover_data=hover_data
)

In [94]:
umap.plot.show(fig)

In [90]:
from sklearn.metrics import silhouette_score

In [91]:
silhouette_score(embs, df['text_output'].apply(lambda x: x.split(': ')[1].split(' > ')[0]), metric='cosine')

0.07854120898663956